<a href="https://colab.research.google.com/github/rcelebi/iswc2020-smarttask/blob/master/TFIDF_type_pred_dbpedia_top_level_type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
path_to_ipynb = '/content/drive/My Drive/iswc-ammar/shervin'
# change directory to where autoregq.py, src, and data are
os.chdir(path_to_ipynb)

In [3]:
import pandas as pd
import numpy as np
dbpedia_df = pd.read_json('data/smarttask_dbpedia_train.json')

In [4]:
test_df= pd.read_json('data/smarttask_dbpedia_test.json')

In [5]:
test_df.head()

,id,question,category,type
0,dbpedia_16015,How many ingredients are in the grain} ?,literal,[number]
1,dbpedia_3885,Is the case fatality rate of Fournier gangrene...,boolean,[boolean]
2,dbpedia_12907,Does the shelf life of spinach equal 8?,boolean,[boolean]
3,dbpedia_7955,What sound does a pig make in the French langu...,literal,[string]
4,dbpedia_2376,When was Fergie completed his record label in ...,literal,[date]


In [6]:
answer_types = {}
for answers in dbpedia_df.type:
    for a in answers:
        if a in answer_types:
            answer_types[a]+=1
        else:
            answer_types[a]=1

def multitype_to_freqtype(type_list):
    for t in sorted(sorted_types, key=lambda item: item[1]):
        if sorted_types[t] < 1000: 
          if t in type_list:
              return t

    for t in sorted(sorted_types, key=lambda item: item[1]):
          if t in type_list:
              return t
    

def types_to_bottom_type(type_list):
    if len(type_list) == 0:
      return None 
    return type_list[0]


def types_to_top_type(type_list):
    if len(type_list) == 0:
      return None 
    return type_list[-1]
            
answ_df =pd.DataFrame.from_dict(answer_types, orient='index', columns=['Freq'])
sorted_types = answ_df.sort_values(by='Freq', ascending=False).to_dict()['Freq']

dbpedia_df['frequent_type'] =dbpedia_df.type.apply(multitype_to_freqtype)
dbpedia_df['bottom_level_type'] =dbpedia_df.type.apply(types_to_bottom_type)
dbpedia_df['top_level_type'] =dbpedia_df.type.apply(types_to_top_type)

In [7]:
test_df['frequent_type'] =test_df.type.apply(multitype_to_freqtype)
test_df['bottom_level_type'] =test_df.type.apply(types_to_bottom_type)
test_df['top_level_type'] =test_df.type.apply(types_to_top_type)

In [8]:
dbpedia_df= dbpedia_df[dbpedia_df.category.notna()]
test_df= test_df[test_df.category.notna()]

In [9]:
dbpedia_df= dbpedia_df[dbpedia_df['frequent_type'].notna()]
test_df= test_df[test_df['frequent_type'].notna()]

In [10]:
dbpedia_df.dropna( subset=['question'], inplace=True)
test_df.dropna( subset=['question'], inplace=True)

In [11]:
dbpedia_df.head()

,id,question,category,type,frequent_type,bottom_level_type,top_level_type
0,dbpedia_1177,Was Jacqueline Kennedy Onassis a follower of M...,boolean,[boolean],boolean,boolean,boolean
1,dbpedia_14427,What is the name of the opera based on Twelfth...,resource,"[dbo:Opera, dbo:MusicalWork, dbo:Work]",dbo:Work,dbo:Opera,dbo:Work
2,dbpedia_16615,When did Lena Horne receive the Grammy Award f...,literal,[date],date,date,date
3,dbpedia_23480,Do Prince Harry and Prince William have the sa...,boolean,[boolean],boolean,boolean,boolean
4,dbpedia_3681,What is the subsidiary company working for Leo...,resource,"[dbo:EducationalInstitution, dbo:Organisation,...",dbo:EducationalInstitution,dbo:EducationalInstitution,dbo:Agent


In [12]:
def extend_cat(row):
  if row['category'] =='boolean':
    return row['category']
  elif row['category'] == 'literal':
    return row['frequent_type']
  else:
    return row['category']


In [13]:
dbpedia_df['extended_cat'] = dbpedia_df.apply(extend_cat, axis=1)

In [14]:
dbpedia_res_df = dbpedia_df[dbpedia_df.category=='resource']

In [15]:
VALIDATION_SPLIT = 0.2
indicies = np.arange(dbpedia_res_df.shape[0])
np.random.shuffle(indicies)
num_valid_samples = int(VALIDATION_SPLIT * dbpedia_res_df.shape[0])
num_valid_samples

1911

In [16]:
num_test_samples=  int(num_valid_samples/2)

In [17]:
dbpedia_res_df=dbpedia_res_df.iloc[indicies]

In [18]:
train_df= dbpedia_res_df[:-num_valid_samples]
train_df.head()

,id,question,category,type,frequent_type,bottom_level_type,top_level_type,extended_cat
11012,dbpedia_7271,What is the largest lake in Canada?,resource,"[dbo:Lake, dbo:BodyOfWater, dbo:NaturalPlace, ...",dbo:NaturalPlace,dbo:Lake,dbo:Location,resource
16135,dbpedia_19025,Name the kingdom with currency Solidus (coin) ...,resource,"[dbo:Country, dbo:PopulatedPlace, dbo:Place, d...",dbo:Country,dbo:Country,dbo:Location,resource
15884,dbpedia_8823,Who is the structural engineer for Burj al-Arab?,resource,"[dbo:Company, dbo:Organisation, dbo:Agent]",dbo:Company,dbo:Company,dbo:Agent,resource
15814,dbpedia_7705,Who is the disciple of Louise Elisabeth Vigee ...,resource,"[dbo:Person, dbo:Agent]",dbo:Agent,dbo:Person,dbo:Agent,resource
8782,dbpedia_20011,What are the television shows whose company is...,resource,"[dbo:TelevisionShow, dbo:Work]",dbo:Work,dbo:TelevisionShow,dbo:Work,resource


In [19]:
valid_df= dbpedia_res_df[-num_valid_samples:]
valid_df.head()

,id,question,category,type,frequent_type,bottom_level_type,top_level_type,extended_cat
10460,dbpedia_17532,Which {film genre} is {voiced by} of {Aestheti...,resource,"[dbo:MusicGenre, dbo:Genre, dbo:TopicalConcept]",dbo:TopicalConcept,dbo:MusicGenre,dbo:TopicalConcept,resource
4377,dbpedia_22304,who are starring in the movies written by Mark...,resource,"[dbo:Person, dbo:Agent]",dbo:Agent,dbo:Person,dbo:Agent,resource
13984,dbpedia_13887,Who is graduated from Georges Dumézil whose pa...,resource,"[dbo:EducationalInstitution, dbo:Organisation,...",dbo:EducationalInstitution,dbo:EducationalInstitution,dbo:Agent,resource
8552,dbpedia_18564,Who all have appointed governess born in Magno...,resource,"[dbo:OfficeHolder, dbo:Person, dbo:Agent]",dbo:OfficeHolder,dbo:OfficeHolder,dbo:Agent,resource
289,dbpedia_18673,To which company is Michael Dell a father comp...,resource,"[dbo:Company, dbo:Organisation, dbo:Agent]",dbo:Company,dbo:Company,dbo:Agent,resource


In [20]:
len(valid_df)

1911

In [21]:
#valid_df.to_csv('/content/drive/My Drive/fastai-v3/data/smarttask/dbpedia_valid_gold.csv', index=False)

In [22]:
#test_df= dbpedia_df[-num_test_samples:]
#test_df.head()

In [23]:
len(valid_df)

1911

## Most frequent/generic type prediction

In [38]:
target = 'top_level_type'


In [39]:
train_res_df= train_df[train_df.category =='resource']
valid_res_df= valid_df[valid_df.category =='resource']
test_res_df= test_df[test_df.category =='resource']
len(train_res_df),len(valid_res_df),len(test_res_df)

(7646, 1911, 2445)

In [40]:
all_res_df = train_res_df.append(valid_res_df).append(test_res_df) 
n_utypes = all_res_df[target].unique()
print (len(n_utypes))

46


In [41]:
len(train_res_df), len(valid_res_df)

(7646, 1911)

In [42]:
type_hierarchy = {}
to_upper_class = {}
for i, row in dbpedia_res_df.type.iteritems():
    #print (row)
    types= list(row)
    for i in range(len(types)-1):
      parent = types[i]
      child = types[i+1]
      #print (parent, child) 
      if parent not in type_hierarchy:
          type_hierarchy[parent] = [child]
      else:
          type_hierarchy[parent].append(child)
          
      if child not in to_upper_class:
          to_upper_class[child] = parent

In [43]:
def to_super_class(specific_type, type_with_less_freq):
    if specific_type in type_with_less_freq:
        if specific_type in to_upper_class:
            #if to_upper_class[specific_type] == 'owl:Thing':
            #   return specific_type
            return to_upper_class[specific_type]
    return specific_type

def to_sub_class(specific_type, type_with_less_freq):
    if specific_type in type_with_less_freq:
        if specific_type in to_upper_class:
            #if to_upper_class[specific_type] == 'owl:Thing':
            #   return specific_type
            return type_hierarchy[parent]
    return specific_type

def choose_freqent_types_balancing(specific_type, iters, min_freq):
    for i in range(iters):
        specific_type_freq = specific_type.value_counts()
        type_with_less_freq = list(specific_type_freq[specific_type_freq < min_freq].keys())
        specific_type = specific_type.apply(lambda x: to_super_class(x, type_with_less_freq) )
    return specific_type

def balance_types(specific_type, min_freq, max_freq):
    # slipt to super-classes if number of 
    
    specific_type_freq = specific_type.value_counts()
    max_freq =  specific_type_freq.sum() * 2  / len(specific_type_freq)
    type_with_most_freq = list(specific_type_freq[specific_type_freq > max_freq].keys())
    type_with_less_freq = list(specific_type_freq[specific_type_freq < min_freq].keys())
    
    specific_type = specific_type.apply(lambda x: to_super_class(x, type_with_less_freq) )


def choose_freqent_types_balancing(specific_type, iters, min_freq):
    for i in range(iters):
        specific_type_freq = specific_type.value_counts()
        type_with_less_freq = list(specific_type_freq[specific_type_freq < min_freq].keys())
        specific_type = specific_type.apply(lambda x: to_super_class(x, type_with_less_freq) )
    return specific_type  

In [44]:
#train_res_df['frequent_type'] =choose_freqent_types_balancing(train_res_df['frequent_type'], 50, 200)

### TF-IDF Text Classifier


In [45]:
label_dict = {}
for index, possible_label in enumerate(sorted(all_res_df[target].unique())):
    label_dict[possible_label] = index
label_dict


{'dbo:Activity': 0,
 'dbo:Agent': 1,
 'dbo:AnatomicalStructure': 2,
 'dbo:Award': 3,
 'dbo:BaseballTeam': 4,
 'dbo:Biomolecule': 5,
 'dbo:ChemicalSubstance': 6,
 'dbo:City': 7,
 'dbo:CollegeCoach': 8,
 'dbo:Country': 9,
 'dbo:Currency': 10,
 'dbo:Device': 11,
 'dbo:Disease': 12,
 'dbo:EthnicGroup': 13,
 'dbo:Event': 14,
 'dbo:Flag': 15,
 'dbo:Food': 16,
 'dbo:Holiday': 17,
 'dbo:HorseRace': 18,
 'dbo:Language': 19,
 'dbo:Location': 20,
 'dbo:MeanOfTransportation': 21,
 'dbo:Media': 22,
 'dbo:MedicalSpecialty': 23,
 'dbo:MountainRange': 24,
 'dbo:Museum': 25,
 'dbo:MusicGenre': 26,
 'dbo:MusicalArtist': 27,
 'dbo:Name': 28,
 'dbo:OfficeHolder': 29,
 'dbo:Person': 30,
 'dbo:PersonFunction': 31,
 'dbo:Place': 32,
 'dbo:PoliticalParty': 33,
 'dbo:PublicService': 34,
 'dbo:RecordLabel': 35,
 'dbo:River': 36,
 'dbo:SoccerClub': 37,
 'dbo:SoccerPlayer': 38,
 'dbo:Species': 39,
 'dbo:TimePeriod': 40,
 'dbo:TopicalConcept': 41,
 'dbo:UnitOfWork': 42,
 'dbo:University': 43,
 'dbo:Work': 44,
 'db

In [46]:
labels_train = train_res_df[target].replace(label_dict).values

labels_val = valid_res_df[target].replace(label_dict).values


In [47]:
labels_test = test_res_df[target].replace(label_dict).values

In [48]:
from sklearn import tree, ensemble
from sklearn.pipeline import Pipeline

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.tree import DecisionTreeClassifier


In [50]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', DecisionTreeClassifier()),
])

In [51]:
pipeline.fit(train_res_df['question'], labels_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
              

In [52]:
pipeline.score(valid_res_df['question'], labels_val)

0.6216640502354788

In [53]:
pipeline.score(test_res_df['question'], labels_test)

0.44785276073619634

## RandomForest Classifier

In [54]:
pipeline1 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', ensemble.RandomForestClassifier(n_estimators=200)),
])

In [55]:
pipeline1.fit(train_res_df['question'], labels_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

In [56]:
pipeline1.score(valid_res_df['question'], labels_val)

0.7022501308215594

In [57]:
pipeline1.score(test_res_df['question'], labels_test)

0.505521472392638

## SVM Classifier

In [58]:
from sklearn.svm import LinearSVC

In [59]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC()),
])

In [60]:
pipeline2.fit(train_res_df['question'], labels_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                     

In [61]:
pipeline2.score(valid_res_df['question'], labels_val)

0.7948717948717948

In [62]:
pipeline2.score(test_res_df['question'], labels_test)

0.5848670756646217